In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timezone, timedelta
from matplotlib import pyplot as plt 
import math
import scipy
import pickle as pkl
import re
import datetime
pd.set_option("display.max_colwidth", None)
import warnings
warnings.filterwarnings('ignore')

C:\Users\20288577529\AppData\Local\Temp\ipykernel_21276\1386116429.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


#### Importamos los DataFrames

In [2]:
# usuarios que mandaron @test_on @test_off @reset_user INCORPORAR LOS NUEVOS INTENT PARA TESTEO QUE SE AGREGARON EN FEBRERO DEL 2024

testers=pd.read_csv('testers.csv')
#testers=testers.f0_.values

In [3]:
rule_ne='PLBWX5XYGQ2B3GP7IN8Q-nml045fna3@b.m-1669990832420'

In [4]:
# Mesage metrics (mensajes)
mm=pd.read_csv('bigQuery/at-mensajes_26F_03M_2024.csv')

In [5]:
# Modificaciones necesesarias al campo creation_time segun la fuente de origen (Bq o Athena)

mm.creation_time=pd.to_datetime(mm.creation_time)

#mm.creation_time = pd.to_datetime(str(mm.creation_time)[10:30])

mm.creation_time=mm.creation_time.dt.tz_localize(None)

#mm = mm.rename(columns = {"vars_value": "value"})

In [6]:
# Validación de fecha mínima o de rango de fecha y hora

mm1=mm[mm['creation_time']>=np.datetime64('2024-02-23 12:30:00') ]
#mm1 = mm[(mm['creation_time'] >= np.datetime64('2024-02-23 17:00:00')) & (mm['creation_time'] <= np.datetime64('2024-02-24 18:00:00'))]

In [7]:
mm1.rule_name.nunique()

4066

In [8]:
# Convierte la columna 'creation_time' de mm1 al formato de fecha y hora.
mm1.creation_time = pd.to_datetime(mm1.creation_time)

# Redondea hacia arriba la columna 'creation_time' al segundo más cercano.
mm1.creation_time = mm1.creation_time.dt.ceil('s')

# Elimina filas duplicadas basadas en las columnas especificadas.
mm1.drop_duplicates(['session_id', 'creation_time', 'msg_from', 'rule_name'], inplace=True)

# Crea una nueva columna 'usuario' que toma los primeros 20 caracteres de la columna 'session_id'.
mm1['usuario'] = mm1.session_id.str[:20]

# Filtra las filas donde el valor de 'usuario' no está en la lista de testers.
mm1 = mm1[~mm1.usuario.isin(testers)]

# Reinicia los índices del DataFrame después de realizar las operaciones anteriores y descarta los índices anteriores.
mm1.reset_index(inplace=True, drop=True)


In [9]:
mm1.usuario.nunique()

169571

In [10]:
# search + response = clicks

# Lee el archivo CSV y carga los datos en el DataFrame 'search'.
search=pd.read_csv('bigQuery/at-clicks_26F_03M_2024.csv')

# Elimina filas duplicadas basadas en las columnas especificadas en el DataFrame 'search'.
search.drop_duplicates(['session_id', 'ts', 'id', 'message', 'mostrado', 'response_message'], inplace=True)

#####################################################
#EN ESTE LUGAR ESTARIA BUENO PROBAR EL MISMO REDONDEO DEL SEGUNDO EN TS PARA EVITAR EL PROBLEMA QUE TUVIMOS CON LA CANTIDAD DE CARACTERAS

# Redondea hacia arriba la columna 'creation_time' al segundo más cercano.
mm1.creation_time = mm1.creation_time.dt.ceil('s')
# Convierte la columna 'ts' (timestamp) de 'search' al formato de fecha y hora.
search.ts=pd.to_datetime(search.ts)
####################################################

# Crea una nueva columna 'usuario' que toma los primeros 20 caracteres de la columna 'session_id'.
search['usuario']=search.session_id.str[:20]

# Filtra las filas donde el valor de 'usuario' no está en la lista de testers.
search=search[~search.usuario.isin(testers)]

# Filtra las filas en las que la columna 'mostrado' es igual a la columna 'response_intent_id' con la condición 'RuleBuilder:'.
# Luego, elimina filas duplicadas basadas en la columna 'id' del resultado y almacena el resultado en 'searchcl'.
searchcl=search['RuleBuilder:'+search.mostrado==search.response_intent_id].drop_duplicates('id')

# Crea una nueva columna 'fecha' en 'search' que contiene solo la parte de la fecha de la columna 'ts'.
search['fecha']=search.ts.dt.date

In [11]:
search.head()

,ts,id,session_id,message_id,message,results_intent_name,results_intent_id,parent_intent_id,parent_intent_name,model_type,results_showable,results_score,rule_id,mostrado,mostrado_name,response_ts,response_message,response_intent_id,usuario,fecha
0,2024-03-03 12:29:57.828,d4f54919-ca61-4c48-929d-31aa3c4cee6e,KKBSO8U3WR652VYE786L_2024-03-03T12:24:55.394Z,SUPTR1Z6IKSUILC4V01F,Natacion en el parque de la ciudad,CU01CUX02 Apertura,PLBWX5XYGQ2B3GP7IN8Q-guadalupe@botmaker.io-1571670970649,NaN,NaN,NaN,True,43.9084,PLBWX5XYGQ2B3GP7IN8Q-6e5jiocf03@b.m-1688135541169,PLBWX5XYGQ2B3GP7IN8Q-guadalupe@botmaker.io-1571670970649,CU01CUX02 Apertura,NaN,NaN,NaN,KKBSO8U3WR652VYE786L,2024-03-03
1,2024-03-03 12:29:57.828,d4f54919-ca61-4c48-929d-31aa3c4cee6e,KKBSO8U3WR652VYE786L_2024-03-03T12:24:55.394Z,SUPTR1Z6IKSUILC4V01F,Natacion en el parque de la ciudad,TR04CUX11 Apertura,PLBWX5XYGQ2B3GP7IN8Q-guadalupe@botmaker.io-1561641844747,NaN,NaN,NaN,True,41.7867,PLBWX5XYGQ2B3GP7IN8Q-6e5jiocf03@b.m-1688135541169,PLBWX5XYGQ2B3GP7IN8Q-guadalupe@botmaker.io-1561641844747,TR04CUX11 Apertura,NaN,NaN,NaN,KKBSO8U3WR652VYE786L,2024-03-03
2,2024-03-03 12:29:57.828,d4f54919-ca61-4c48-929d-31aa3c4cee6e,KKBSO8U3WR652VYE786L_2024-03-03T12:24:55.394Z,SUPTR1Z6IKSUILC4V01F,Natacion en el parque de la ciudad,CU05CUX01 Apertura,PLBWX5XYGQ2B3GP7IN8Q-4pefum0myw@b.m-1631643781450,NaN,NaN,NaN,True,41.9892,PLBWX5XYGQ2B3GP7IN8Q-6e5jiocf03@b.m-1688135541169,PLBWX5XYGQ2B3GP7IN8Q-4pefum0myw@b.m-1631643781450,CU05CUX01 Apertura,NaN,NaN,NaN,KKBSO8U3WR652VYE786L,2024-03-03
3,2024-03-03 12:29:57.828,d4f54919-ca61-4c48-929d-31aa3c4cee6e,KKBSO8U3WR652VYE786L_2024-03-03T12:24:55.394Z,SUPTR1Z6IKSUILC4V01F,Natacion en el parque de la ciudad,CIU05CUX03 Parque Patricios,PLBWX5XYGQ2B3GP7IN8Q-6a5d6b844e29152@bm.com-1638297083965,NaN,NaN,NaN,True,62.9072,PLBWX5XYGQ2B3GP7IN8Q-6e5jiocf03@b.m-1688135541169,PLBWX5XYGQ2B3GP7IN8Q-6a5d6b844e29152@bm.com-1638297083965,CIU05CUX03 Parque Patricios,NaN,NaN,NaN,KKBSO8U3WR652VYE786L,2024-03-03
4,2024-03-03 12:29:57.828,d4f54919-ca61-4c48-929d-31aa3c4cee6e,KKBSO8U3WR652VYE786L_2024-03-03T12:24:55.394Z,SUPTR1Z6IKSUILC4V01F,Natacion en el parque de la ciudad,TR02CUX14 Apertura por Whatsapp,PLBWX5XYGQ2B3GP7IN8Q-p7o7yuoq7i@b.m-1688998686012,NaN,NaN,NaN,True,62.8261,PLBWX5XYGQ2B3GP7IN8Q-6e5jiocf03@b.m-1688135541169,PLBWX5XYGQ2B3GP7IN8Q-p7o7yuoq7i@b.m-1688998686012,TR02CUX14 Apertura por Whatsapp,NaN,NaN,NaN,KKBSO8U3WR652VYE786L,2024-03-03


In [12]:
searchcl.head()

,ts,id,session_id,message_id,message,results_intent_name,results_intent_id,parent_intent_id,parent_intent_name,model_type,results_showable,results_score,rule_id,mostrado,mostrado_name,response_ts,response_message,response_intent_id,usuario
83,2024-03-03 12:29:44.996,48e1c827-9925-499c-a08d-53c26309b685,82A3THWO2G2M6VP1M6HO_2024-03-03T12:29:44.216Z,7RV5I8Z7IPIHMAF72EQX,"¡Hola, Boti! Quiero un turno médico. �",TUR00CUX02 Mensaje inicial - Turnos,PLBWX5XYGQ2B3GP7IN8Q-rodrigo@botmaker.io-1536765072955,NaN,NaN,NaN,True,78.4655,PLBWX5XYGQ2B3GP7IN8Q-6e5jiocf03@b.m-1688135541169,PLBWX5XYGQ2B3GP7IN8Q-rodrigo@botmaker.io-1536765072955,TUR00CUX02 Mensaje inicial - Turnos,2024-03-03 12:29:53.798,"{""button"":""Otros turnos"",""entities"":""{}"",""originalMessage"":""C"",""intent"":""RuleBuilder:PLBWX5XYGQ2B3GP7IN8Q-rodrigo@botmaker.io-1536765072955""}",RuleBuilder:PLBWX5XYGQ2B3GP7IN8Q-rodrigo@botmaker.io-1536765072955,82A3THWO2G2M6VP1M6HO
140,2024-03-03 12:29:38.489,0ac9017a-f460-4f32-bbfe-46fb35314f03,0SKKYJ7Z4JAPP074Q0PS_2024-03-03T12:27:11.694Z,WS1JA6WZ35IHHTCLK0AB,X,TR04CUX23 Apertura,PLBWX5XYGQ2B3GP7IN8Q-uq5ih4cjc9@b.m-1661868287316,NaN,NaN,NaN,True,79.5030,PLBWX5XYGQ2B3GP7IN8Q-y272iwgx5g@b.m-1688392737934,PLBWX5XYGQ2B3GP7IN8Q-uq5ih4cjc9@b.m-1661868287316,TR04CUX23 Apertura,2024-03-03 12:29:45.021,"{""button"":""Boleta BUI"",""entities"":""{}"",""originalMessage"":""B"",""intent"":""RuleBuilder:PLBWX5XYGQ2B3GP7IN8Q-uq5ih4cjc9@b.m-1661868287316""}",RuleBuilder:PLBWX5XYGQ2B3GP7IN8Q-uq5ih4cjc9@b.m-1661868287316,0SKKYJ7Z4JAPP074Q0PS
239,2024-03-03 12:29:26.595,3c331439-912c-4b4d-bd8d-e713b48567fd,B7NAKLB840YR77DPBLS0_2024-03-03T12:27:19.054Z,IR61M4Z0TSOVRSAKNILU,denuncia vial,SUACI - Reintentar API - Denuncia vial,PLBWX5XYGQ2B3GP7IN8Q-dario@botmaker.io-1589293768116,PLBWX5XYGQ2B3GP7IN8Q-e6f14faf47ab40f@bm.com-1638209334091,SUA01CUX00 Apertura,NaN,False,100.0541,PLBWX5XYGQ2B3GP7IN8Q-6e5jiocf03@b.m-1688135541169,PLBWX5XYGQ2B3GP7IN8Q-e6f14faf47ab40f@bm.com-1638209334091,SUA01CUX00 Apertura,2024-03-03 12:29:43.214,"{""button"":""Solicitudes 👷"",""entities"":""{}"",""originalMessage"":""A"",""intent"":""RuleBuilder:PLBWX5XYGQ2B3GP7IN8Q-e6f14faf47ab40f@bm.com-1638209334091""}",RuleBuilder:PLBWX5XYGQ2B3GP7IN8Q-e6f14faf47ab40f@bm.com-1638209334091,B7NAKLB840YR77DPBLS0
388,2024-03-03 12:28:58.790,091d121c-cdd3-4b71-99bb-b22eea46bd92,YECXKDA64WWYT5Q6GSK7_2024-03-03T12:26:58.189Z,YxzMdOrjLkViCoK3YMEv,Vacuna covid,SA03CUX01 Vacunas por edad,PLBWX5XYGQ2B3GP7IN8Q-guadalupe@botmaker.io-1577115780647,PLBWX5XYGQ2B3GP7IN8Q-guadalupe@botmaker.io-1578061373626,SA03CUX01 Buscar vacunatorios,NaN,False,57.8826,PLBWX5XYGQ2B3GP7IN8Q-6e5jiocf03@b.m-1688135541169,PLBWX5XYGQ2B3GP7IN8Q-guadalupe@botmaker.io-1578061373626,SA03CUX01 Buscar vacunatorios,2024-03-03 12:29:15.097,"{""intent"":""RuleBuilder:PLBWX5XYGQ2B3GP7IN8Q-guadalupe@botmaker.io-1578061373626"",""button"":""Buscar vacunatorios"",""entities"":""{}""}",RuleBuilder:PLBWX5XYGQ2B3GP7IN8Q-guadalupe@botmaker.io-1578061373626,YECXKDA64WWYT5Q6GSK7
540,2024-03-03 12:28:33.225,49ccb5dd-8d07-42f2-82da-6d7b5d87f061,GNM3GKXLHCXEJX6W5BGB_2024-03-03T12:21:01.544Z,TMFN8DsNF5sY6mefXM6d,Talleres,CU11CUX02 Apertura,PLBWX5XYGQ2B3GP7IN8Q-ar19982fo6@b.m-1692031764158,NaN,NaN,NaN,True,55.5698,PLBWX5XYGQ2B3GP7IN8Q-6e5jiocf03@b.m-1688135541169,PLBWX5XYGQ2B3GP7IN8Q-ar19982fo6@b.m-1692031764158,CU11CUX02 Apertura,2024-03-03 12:28:39.870,"{""button"":""Talleres culturales"",""entities"":""{}"",""intent"":""RuleBuilder:PLBWX5XYGQ2B3GP7IN8Q-ar19982fo6@b.m-1692031764158""}",RuleBuilder:PLBWX5XYGQ2B3GP7IN8Q-ar19982fo6@b.m-1692031764158,GNM3GKXLHCXEJX6W5BGB


In [13]:
# user-buttons (botones)
one=pd.read_csv('bigQuery/at-botones_26F_03M_2024.csv')

In [14]:
# Procesamiento de datos en el DataFrame 'one':  ONESHOTS

# Crea una nueva columna 'usuario' en el DataFrame 'one', que toma los primeros 20 caracteres de la columna 'session_id'.
one['usuario'] = one.session_id.str[:20]

# Filtra las filas donde el valor de 'usuario' no está en la lista de testers.
one = one[~one.usuario.isin(testers)]

# Filtra las filas en el DataFrame 'one' donde 'one_shot' es True y 'type' es 'oneShot' o 'oneShotSearch'.
os = one[np.logical_and(one.one_shot == True, one.type.isin(['oneShot', 'oneShotSearch']))]

# Convierte la columna 'ts' (timestamp) de 'os' al formato de fecha y hora. HABRIA QUE REDONDEAR LSO SEGUNDOS POR LAS DUDAS TAMBIEN?
os.ts = pd.to_datetime(os.ts)

# Crea una nueva columna 'fecha' en 'os' que contiene solo la parte de la fecha de la columna 'ts'.
os['fecha'] = os.ts.dt.date


In [15]:
#importamos el csv con la lista de los intents mostrables (tambien aparece como showable)

mos=pd.read_csv('Actualizacion Lista Blanca - 2023 - Whitelist intents 17_07.csv')
# Elimina los espacios en blanco alrededor de los nombres de intenciones en la columna 'Nombre de la intención'.
rules_mos = mos['Nombre de la intención'].str.strip().values

### Transformaciones

In [16]:
# sacamos mensajes seguidos de boti

# Reinicia los índices del DataFrame 'mm1', descartando los índices anteriores y aplicando los cambios en el lugar.
mm1.reset_index(inplace=True, drop=True)

# Crea una lista 'drop' que contiene los índices de las filas que deben eliminarse.
drop = [i if mm1.loc[i].msg_from == mm1.loc[i+1].msg_from and mm1.loc[i].session_id == mm1.loc[i+1].session_id else None for i in mm1.index[:-1]]

# Convierte la lista 'drop' en un conjunto para eliminar duplicados y luego convierte de nuevo a lista.
drop = list(set(drop))

# Elimina los valores 'None' de la lista 'drop'.
drop.remove(None)

# Elimina las filas del DataFrame 'mm1' utilizando los índices almacenados en la lista 'drop'.
mm1.drop(drop, inplace=True)

# Reinicia los índices del DataFrame 'mm1' después de eliminar las filas, descartando los índices anteriores y aplicando los cambios en el lugar.
mm1.reset_index(inplace=True, drop=True)


In [17]:
# Selecciona las filas en el DataFrame 'mm1' donde la columna 'max_score' no es nula y muestra las primeras filas.
mm1[~mm1['max_score'].isnull()].head()


,session_id,id,creation_time,msg_from,message_type,message,rule_name,topic_path,original_user_message,max_score,vars_value,usuario
23,00227WI6GFFS7HBRKW37_2024-02-28T01:10:09.030Z,4V7PMFE55DT00P0BZL16,2024-02-28 01:11:05,user,Text,Desratización,NaN,NaN,NaN,41.3302,5.0,00227WI6GFFS7HBRKW37
53,002G64V6MLKUVFZSY5HD_2024-02-27T19:57:54.115Z,ACJ36H02AIIWC5SH78IA,2024-02-27 19:58:13,user,Text,Turnos hospital,NaN,NaN,NaN,94.2484,190.0,002G64V6MLKUVFZSY5HD
71,002G64V6MLKUVFZSY5HD_2024-02-27T19:57:54.115Z,SVHTHV7W4NUD1SWRJY0K,2024-02-27 20:14:16,user,Text,Turno traumotologia,NaN,NaN,NaN,59.0226,190.0,002G64V6MLKUVFZSY5HD
79,002G64V6MLKUVFZSY5HD_2024-02-27T19:57:54.115Z,8E2E2CWBFIB624KIMZ4E,2024-02-27 20:15:34,user,Text,Hospital publico,NaN,NaN,NaN,155.0368,190.0,002G64V6MLKUVFZSY5HD
125,002LM1QGYP57KHSMUJNW_2024-02-28T23:50:23.723Z,FPDEAZ1KAD1W6NIEB2OM,2024-02-28 23:52:19,user,Text,Poda,NaN,NaN,NaN,59.7098,178.0,002LM1QGYP57KHSMUJNW


#### Modelo Nuevo Primera Instancia

In [18]:
# Análisis de respuestas por usuario en el DataFrame 'mm1':


# Copia y reinicia los índices del DataFrame 'mm1'.
mm = mm1.copy()
mm.reset_index(inplace=True, drop=True)

# Filtra y selecciona columnas específicas de mensajes de texto enviados por usuarios.
mmtex1 = mm[np.logical_and(mm.msg_from == 'user', mm.message_type == 'Text')][['session_id', 'id', 'creation_time', 'msg_from', 'message_type', 'message', 'usuario']]

# Crea la columna 'rule_name' basada en ciertas condiciones.
mmtex1['rule_name'] = [r if su == sb and f == 'bot' else None for r, su, sb, f in zip(mm.loc[mmtex1.index + 1].rule_name.values, mmtex1.session_id.values, mm.loc[mmtex1.index + 1].session_id.values, mm.loc[mmtex1.index + 1].msg_from.values)]

# Filtra y selecciona las filas relacionadas con la regla 'No entendió letra no existente en WA'.
letra1 = mmtex1[mmtex1.rule_name == 'No entendió letra no existente en WA']
letra1.rename(columns={'id': 'message_id'}, inplace=True)

# Filtra 'search' y 'os' según 'mm1.session_id.values'.
search1 = search[search.session_id.isin(mm1.session_id.values)]
os1 = os[os.session_id.isin(mm1.session_id.values)]

# Filtra las instancias iniciales que no están en 'search1' o 'os1' y realiza algunas transformaciones.
primera_instancia1 = search[~search.message_id.isin(pd.concat([search1['RuleBuilder:' + search1.mostrado == search1.response_intent_id].message_id, os1.message_id]).values)].drop_duplicates('id')
primera_instancia1.rename(columns={"results_score": "score"}, inplace=True)
ne1 = primera_instancia1.groupby('id').max()[['session_id', 'message_id', 'score']]
ne1 = ne1[ne1.score <= 5.36]

primera_instancia1 = primera_instancia1[~primera_instancia1.id.isin(ne1.index)]
os1 = os1.drop_duplicates('id')[['session_id', 'message_id']]
click1 = search1['RuleBuilder:' + search1.mostrado == search1.response_intent_id].drop_duplicates('id')[['session_id', 'message_id']]
abandonos1 = primera_instancia1[primera_instancia1.response_message.isna()][['session_id', 'message_id']]
nada1 = primera_instancia1[primera_instancia1.response_intent_id == 'RuleBuilder:PLBWX5XYGQ2B3GP7IN8Q-alfafc@gmail.com-1536777380652'][['session_id', 'message_id']]
texto1 = primera_instancia1[np.logical_and(primera_instancia1.response_intent_id != 'RuleBuilder:PLBWX5XYGQ2B3GP7IN8Q-alfafc@gmail.com-1536777380652', ~primera_instancia1.response_message.isna())][['session_id', 'message_id']]
letra1 = letra1[['session_id', 'message_id']]

# Agrega una columna 'categoria' a 'os1', 'click1', 'abandonos1', 'nada1', 'texto1', 'ne1', y 'letra1'.
os1['categoria'] = 'one'
click1['categoria'] = 'click'
abandonos1['categoria'] = 'abandono'
nada1['categoria'] = 'nada'
texto1['categoria'] = 'texto'
ne1['categoria'] = 'ne'
letra1['categoria'] = 'letra'

# Concatena y filtra 'value1primera' según 'mm1.usuario.values'.
value1primera = pd.concat([os1, click1, abandonos1, nada1, texto1, ne1, letra1])
value1primera['usuario'] = value1primera.session_id.str[:20]
value1primera = value1primera[value1primera.usuario.isin(mm1.usuario.values)]

# Realiza un análisis de respuestas por usuario y categoría.
respuestas_por_usuario = value1primera.groupby(['usuario', 'categoria'], as_index=False).count()[['usuario', 'categoria', 'message_id']].pivot_table('message_id', ['usuario'], 'categoria')
respuestas_por_usuario.fillna(0, inplace=True)
respuestas_por_usuario = respuestas_por_usuario.reset_index(drop=False).reindex(['usuario', 'one', 'click', 'texto', 'abandono', 'nada', 'ne', 'letra'], axis=1)

# Calcula porcentajes por categoría para cada usuario.
respuestas_por_usuario['porcentaje_abandono']=[respuestas_por_usuario.loc[i].abandono / respuestas_por_usuario.loc[i][['one', 'click', 'texto', 'abandono', 'nada', 'ne', 'letra']].sum() for i in respuestas_por_usuario.index]
respuestas_por_usuario['porcentaje_click']=[respuestas_por_usuario.loc[i].click / respuestas_por_usuario.loc[i][['one', 'click', 'texto', 'abandono', 'nada', 'ne', 'letra']].sum() for i in respuestas_por_usuario.index]
respuestas_por_usuario['porcentaje_one']=[respuestas_por_usuario.loc[i].one / respuestas_por_usuario.loc[i][['one', 'click', 'texto', 'abandono', 'nada', 'ne', 'letra']].sum() for i in respuestas_por_usuario.index]
respuestas_por_usuario['porcentaje_texto']=[respuestas_por_usuario.loc[i].texto / respuestas_por_usuario.loc[i][['one', 'click', 'texto', 'abandono', 'nada', 'ne', 'letra']].sum() for i in respuestas_por_usuario.index]
respuestas_por_usuario['porcentaje_nada']=[respuestas_por_usuario.loc[i].nada / respuestas_por_usuario.loc[i][['one', 'click', 'texto', 'abandono', 'nada', 'ne', 'letra']].sum() for i in respuestas_por_usuario.index]
respuestas_por_usuario['porcentaje_ne']=[respuestas_por_usuario.loc[i]['ne'] / respuestas_por_usuario.loc[i][['one', 'click', 'texto', 'abandono', 'nada', 'ne', 'letra']].sum() for i in respuestas_por_usuario.index]
respuestas_por_usuario['porcentaje_letra']=[respuestas_por_usuario.loc[i]['letra'] / respuestas_por_usuario.loc[i][['one', 'click', 'texto', 'abandono', 'nada', 'ne', 'letra']].sum() for i in respuestas_por_usuario.index]


# Crea un DataFrame 'res_primera_instancia1' basado en 'respuestas_por_usuario'.
res_primera_instancia1 = respuestas_por_usuario.copy()

# Calcula promedios para diferentes categorías.
promedios1={'abandonos': round(respuestas_por_usuario['porcentaje_abandono'].mean(), 3),     
                  'click': round(respuestas_por_usuario['porcentaje_click'].mean(), 3),
                  'one': round(respuestas_por_usuario['porcentaje_one'].mean(), 3),
                  'texto': round(respuestas_por_usuario['porcentaje_texto'].mean(), 3),
                  'nada': round(respuestas_por_usuario['porcentaje_nada'].mean(), 3),
                  'letra': round(respuestas_por_usuario['porcentaje_letra'].mean(), 3),
                  'ne': round(respuestas_por_usuario['porcentaje_ne'].mean(), 3)
}



In [19]:
promedios1

{'abandonos': 0.046,
 'click': 0.197,
 'one': 0.58,
 'texto': 0.033,
 'nada': 0.055,
 'letra': 0.076,
 'ne': 0.013}

In [20]:
# Definimos la funcion que identifica las distintas categorías

def categoria(m, t, r):
    # Esta función categoriza mensajes basándose en el contenido del mensaje (m), el tipo de mensaje (t), y el nombre de la regla (r).

    try:
        # Verifica si el tipo de mensaje es 'Button-click' y si el mensaje contiene 'Cambiar de tema'.
        if t == 'Button-click' and 'Cambiar de tema' in m:
            return 'cambiar'

        # Verifica si el tipo de mensaje es 'Button-click' y si la regla es 'Menú show buttons'.
        elif t == 'Button-click' and r == 'Menú show buttons':
            return 'otros'

        # Verifica si el tipo de mensaje es 'Button-click' y si el mensaje contiene 'No era nada de eso'.
        elif t == 'Button-click' and 'No era nada de eso' in m:
            return 'x'

        # Verifica si el tipo de mensaje es 'Button-click'.
        elif t == 'Button-click':
            return 'boton'

        # Verifica si el mensaje es 'a', 'b', 'c', o 'd' (ignorando mayúsculas y minúsculas) y si la regla es 'Infracciones * Apertura'.
        elif re.match(r'^a$|^b$|^c$|^d$', m, re.IGNORECASE) and r == 'Infracciones * Apertura':
            return 'boton'

        # Verifica si el mensaje es 'a', 'b', 'c', o 'd' (ignorando mayúsculas y minúsculas) y si la regla es 'Busca donde está permitido estacionar'.
        elif re.match(r'^a$|^b$|^c$|^d$', m, re.IGNORECASE) and r == 'Busca donde está permitido estacionar':
            return 'boton'

        # elif m == '__image__' and r == 'Denuncia Vial - Validación Vehículo':
        #     return 'boton'
        # elif re.match(r'[0-9]{7,8}', m) and r == 'Licencia prorroga  > Consultar':
        #     return 'boton'

        # Verifica si el mensaje es 'x' (ignorando mayúsculas y minúsculas) o 'x buscaba otra cosa'.
        elif re.match(r'(^x$)|(x?\.? ?buscaba otra cosa)', m, re.IGNORECASE):
            return 'x'

        # Si no coincide con ninguna de las condiciones anteriores, categoriza como 'texto'.
        else:
            return 'texto'

    except:
        # Maneja excepciones y retorna 'otros' en caso de error.
        return 'otros'


In [21]:
# Análisis de interacciones del usuario en el DataFrame 'mm1':

# Copia el DataFrame 'mm1' a 'mm'.
mm = mm1.copy()

# Reinicia los índices de 'mm', descartando los índices anteriores y aplicando los cambios en el lugar.
mm.reset_index(inplace=True, drop=True)

# Filtra las filas donde 'msg_from' es 'user'.
mmu = mm[mm.msg_from == 'user']

# Reinicia los índices de 'mmu', descartando los índices anteriores y aplicando los cambios en el lugar.
mmu.reset_index(inplace=True, drop=True)

# Filtra las filas originales del usuario que están en 'searchcl.message_id'.
original = mmu[mmu.id.isin(searchcl.message_id.values)] 

# Obtiene las filas siguientes (botones) y respuestas subsiguientes.
boton = mmu.loc[original.index + 1]
respuesta = mmu.loc[original.index + 2]

# Crea un DataFrame 'conv_cl' con información de la conversación, como la sesión, la hora de creación y mensajes originales.
# También incluye información sobre el intent, el primer botón, respuestas intermedias y finales.
conv_cl = pd.DataFrame(data={'session_id': original.session_id.values, 'creation_time': original.creation_time.values, 'original': original.message.values, 
                             'intent': mm.loc[mm[mm.id.isin(boton.id.values)].index + 1].rule_name.values,
                             'bot1_id': [m if v else None for m, v in zip(mm.loc[mm[mm.id.isin(boton.id.values)].index + 1].id.values, original.session_id.values==mm.loc[mm[mm.id.isin(boton.id.values)].index + 1].session_id.values)],
                             'respuesta_intermedia': [m if v else None for m, v in zip(boton.message.values, original.session_id.values==boton.session_id.values)], 
                             'respuesta': [m if v else None for m, v in zip(respuesta.message.values, original.session_id.values==respuesta.session_id.values)],
                             'respuesta_type': [m if v else None for m, v in zip(respuesta.message_type.values, original.session_id.values==respuesta.session_id.values)],
                             'respuesta_rule': [m if v else None for m, v in zip(mm.loc[mm[mm.id.isin(respuesta.id.values)].index + 1].rule_name.values, original.session_id.values==mm.loc[mm[mm.id.isin(respuesta.id.values)].index + 1].session_id.values)]})

mm = mm1.copy()
mm.reset_index(inplace=True, drop=True)
mmu = mm[mm.msg_from == 'user']
mmu.reset_index(inplace=True, drop=True)

# Filtra las filas originales del usuario que están en 'os.message_id'.
original = mmu[mmu.id.isin(os.message_id.values)]

# Obtiene la respuesta subsiguiente.
respuesta = mmu.loc[original.index + 1]

# Crea un DataFrame 'conv' con información de la conversación, como la sesión, la hora de creación y mensajes originales.
# También incluye información sobre el intent, el primer botón, y respuestas subsiguientes.
conv = pd.DataFrame(data={'session_id': original.session_id.values, 'creation_time': original.creation_time.values, 'original': original.message.values, 
                          'intent': mm.loc[mm[mm.id.isin(original.id.values)].index + 1].rule_name.values,
                          'bot1_id': [m if v else None for m, v in zip(mm.loc[mm[mm.id.isin(original.id.values)].index + 1].id.values, original.session_id.values==mm.loc[mm[mm.id.isin(original.id.values)].index + 1].session_id.values)],
                          'respuesta': [m if v else None for m, v in zip(respuesta.message.values, original.session_id.values==respuesta.session_id.values)],
                          'respuesta_type': [m if v else None for m, v in zip(respuesta.message_type.values, original.session_id.values==respuesta.session_id.values)],
                          'respuesta_rule': [m if v else None for m, v in zip(mm.loc[mm[mm.id.isin(respuesta.id.values)].index + 1].rule_name.values, original.session_id.values==mm.loc[mm[mm.id.isin(respuesta.id.values)].index + 1].session_id.values)]})


# Crea una nueva columna 'categoria' en 'conv' utilizando la función 'categoria'.
conv['categoria'] = [categoria(m, t, r) if m is not None else 'abandono' for m, t, r in zip(conv.respuesta, conv.respuesta_type, conv.intent)]

# Agrupa por 'categoria' y cuenta la cantidad de 'bot1_id' para cada categoría.
per = conv.groupby('categoria', as_index=False).count()[['categoria', 'bot1_id']]

# Calcula el porcentaje de cada categoría respecto al total.
per['per'] = per.bot1_id / per.bot1_id.sum()

# Concatena 'conv_cl' y 'conv' en un nuevo DataFrame 'usuario1', y agrega columnas adicionales.
usuario1 = pd.concat([conv_cl[['session_id', 'creation_time', 'original', 'intent', 'bot1_id', 'respuesta', 'respuesta_type', 'respuesta_rule']], conv])

# Crea una nueva columna 'categoria' en 'usuario1' utilizando la función 'categoria'.
usuario1['categoria'] = [categoria(m, t, r) if m is not None else 'abandono' for m, t, r in zip(usuario1.respuesta, usuario1.respuesta_type, usuario1.intent)]

# Crea una nueva columna 'usuario' en 'usuario1' extrayendo los primeros 20 caracteres de 'session_id'.
usuario1['usuario'] = usuario1.session_id.str[:20]

# Crea una nueva columna 'id' en 'usuario1' copiando los valores de 'bot1_id'.
usuario1['id'] = usuario1.bot1_id



#### Resultados

In [22]:
# Análisis de resultados por usuario:

resultados=[]
promedios=[]

# Itera sobre el DataFrame 'usuario1'.
for usuario in [usuario1]:

    # Agrupa por 'usuario' y 'categoria', y cuenta la cantidad de 'id' para cada categoría.
    respuestas_por_usuario=usuario.groupby(['usuario','categoria'], as_index=False).count()[['usuario','categoria', 'id']].pivot_table('id', ['usuario'], 'categoria')
    respuestas_por_usuario.fillna(0, inplace=True)

    # Reordena las columnas del DataFrame 'respuestas_por_usuario'.
    respuestas_por_usuario=respuestas_por_usuario.reset_index(drop=False).reindex(['usuario', 'abandono', 'boton', 'otros', 'texto', 'x', 'cambiar'], axis=1)

    # Calcula los porcentajes para cada categoría.
    respuestas_por_usuario['porcentaje_abandono']=[respuestas_por_usuario.loc[i].abandono / respuestas_por_usuario.loc[i][['abandono', 'boton', 'otros', 'texto', 'x',  'cambiar']].sum() for i in respuestas_por_usuario.index]
    respuestas_por_usuario['porcentaje_boton']=[respuestas_por_usuario.loc[i].boton / respuestas_por_usuario.loc[i][['abandono', 'boton', 'otros', 'texto', 'x',  'cambiar']].sum() for i in respuestas_por_usuario.index]
    respuestas_por_usuario['porcentaje_otros']=[respuestas_por_usuario.loc[i].otros / respuestas_por_usuario.loc[i][['abandono', 'boton', 'otros', 'texto', 'x',  'cambiar']].sum() for i in respuestas_por_usuario.index]
    respuestas_por_usuario['porcentaje_texto']=[respuestas_por_usuario.loc[i].texto / respuestas_por_usuario.loc[i][['abandono', 'boton', 'otros', 'texto', 'x',  'cambiar']].sum() for i in respuestas_por_usuario.index]
    respuestas_por_usuario['porcentaje_x']=[respuestas_por_usuario.loc[i].x / respuestas_por_usuario.loc[i][['abandono', 'boton', 'otros', 'texto', 'x',  'cambiar']].sum() for i in respuestas_por_usuario.index]
    respuestas_por_usuario['porcentaje_cambiar']=[respuestas_por_usuario.loc[i].cambiar / respuestas_por_usuario.loc[i][['abandono', 'boton', 'otros', 'texto', 'x',  'cambiar']].sum() for i in respuestas_por_usuario.index]

    # Agrega el DataFrame 'respuestas_por_usuario' a la lista 'resultados'.
    resultados.append(respuestas_por_usuario)

    # Calcula los promedios de porcentajes para cada categoría y agrega a la lista 'promedios'.
    promedios.append({'abandonos': round(respuestas_por_usuario['porcentaje_abandono'].mean(), 3),     
                      'botones': round(respuestas_por_usuario['porcentaje_boton'].mean(), 3),
                      'otros': round(respuestas_por_usuario['porcentaje_otros'].mean(), 3),
                      'texto': round(respuestas_por_usuario['porcentaje_texto'].mean(), 3),
                      'x': round(respuestas_por_usuario['porcentaje_x'].mean(), 3),
                      'cambiar de tema': round(respuestas_por_usuario['porcentaje_cambiar'].mean(), 3)})



In [23]:
pd.DataFrame(promedios, index=['nuevo-con-oss', 'nuevo-sin-oss'])[['abandonos', 'botones', 'texto', 'x', 'cambiar de tema', 'otros']]

,abandonos,botones,texto,x,cambiar de tema,otros
nuevo-con-oss,0.092,0.785,0.08,0.019,0.023,0.0
nuevo-sin-oss,0.092,0.785,0.08,0.019,0.023,0.0


In [24]:
# Crea un nuevo diccionario 'con_oss1' multiplicando cada valor en 'promedios1' por 100.
con_oss1 = {k: v * 100 for k, v in promedios1.items()}

# Crea un nuevo diccionario 'con_oss2' multiplicando los valores correspondientes en 'promedios1' por las sumas ponderadas de 'click' y 'one' en 'promedios'.
con_oss2 = {k: (promedios1['click'] + promedios1['one']) * v * 100 for k, v in promedios[0].items()}


In [25]:
con_oss1

{'abandonos': 4.6,
 'click': 19.7,
 'one': 57.99999999999999,
 'texto': 3.3000000000000003,
 'nada': 5.5,
 'letra': 7.6,
 'ne': 1.3}

In [26]:
con_oss2

{'abandonos': 7.148399999999999,
 'botones': 60.994499999999995,
 'otros': 0.0,
 'texto': 6.215999999999999,
 'x': 1.4762999999999997,
 'cambiar de tema': 1.7870999999999997}